In [1]:
import sqlite3

In [2]:
connection = sqlite3.connect("testdb.db")
connection.close()

In [3]:
sqlite3.version

'2.6.0'

In [4]:
sqlite3.sqlite_version

'3.32.3'

In [13]:
connection = sqlite3.connect("testdb.db")

with connection:
#     with connection.cursor() as currsor: 
    cursor = connection.cursor()  
    cursor.execute('SELECT SQLITE_VERSION()')
    data = cursor.fetchone()
    print("SQLite version: %s" % data)
    cursor.close()

SQLite version: 3.32.3


In [14]:
cars = (
    (1, 'Audi', 52642),
    (2, 'Mercedes', 57127),
    (3, 'Skoda', 9000),
    (4, 'Volvo', 29000),
    (5, 'Bentley', 350000),
    (6, 'Hummer', 41400),
    (7, 'Volkswagen', 21600)
)
con = sqlite3.connect("testdb.db")
 
with con:
    cur = con.cursor()
    cur.execute("DROP TABLE IF EXISTS Cars")
    cur.execute("CREATE TABLE Cars(Id INT, Name TEXT, Price INT)")
    cur.executemany("INSERT INTO Cars VALUES(?, ?, ?)", cars)

In [25]:
with con:
    cur.execute("SELECT Name, Price FROM Cars WHERE Price > 50000")
    data = cur.fetchall()
    for row in data:
#         print(f"{row[0]:<10s} | {row[1]:6d}")
#         print(f"{row[0]:2d} | {row[1]:<10s} | {row[2]:6d}")

Audi       |  52642
Mercedes   |  57127
Bentley    | 350000


In [28]:
with con:
    cur.execute("SELECT * FROM Cars")
    data = cur.fetchall()
    for row in data:
#         print(f"{row[0]:<10s} | {row[1]:6d}")
        print(f"{row[0]:2d} | {row[1]:<10s} | {row[2]:6d}")

 1 | Audi       |  62300
 2 | Mercedes   |  57127
 3 | Skoda      |   9000
 4 | Volvo      |  29000
 5 | Bentley    | 350000
 6 | Hummer     |  41400
 7 | Volkswagen |  21600


In [27]:
uId = 1
uPrice = 62300 
with con:
    cur = con.cursor()    
    cur.execute("UPDATE Cars SET Price=? WHERE Id=?", (uPrice, uId))        
    con.commit()
    print("Number of rows updated: {}".format(cur.rowcount))

Number of rows updated: 1


In [29]:
con.close()

In [30]:
from sqlalchemy import create_engine

In [31]:
engine = create_engine("sqlite:///test_alchemy.db")

In [32]:
engine

Engine(sqlite:///test_alchemy.db)

In [33]:
conn = engine.connect()
conn.execute("CREATE TABLE Cars(Id INT, Name TEXT, Price INT)")
# result = conn.execute("select * from employee")
# result.fetchall()
conn.close()
#     cur.execute("DROP TABLE IF EXISTS Cars")
#     cur.execute("CREATE TABLE Cars(Id INT, Name TEXT, Price INT)")
#     cur.executemany("INSERT INTO Cars VALUES(?, ?, ?)", cars)

In [34]:
cars = (
    (1, 'Audi', 52642),
    (2, 'Mercedes', 57127),
    (3, 'Skoda', 9000),
    (4, 'Volvo', 29000),
    (5, 'Bentley', 350000),
    (6, 'Hummer', 41400),
    (7, 'Volkswagen', 21600)
)

In [35]:
conn = engine.connect()
trans = conn.begin()
# conn.execute("insert into employee (emp_name) values (:emp_name)", emp_name="wendy")
conn.execute("INSERT INTO Cars VALUES(:idv, :name, :price)", idv=1, name='Audi', price=52642)
trans.commit()
conn.close()

In [36]:
conn = engine.connect()
trans = conn.begin()
# conn.execute("insert into employee (emp_name) values (:emp_name)", emp_name="wendy")
for idv_, name_, price_ in cars[1:]:
    conn.execute("INSERT INTO Cars VALUES(:idv, :name, :price)", idv=idv_, name=name_, price=price_)
trans.commit()
conn.close()

In [43]:
engine = create_engine("sqlite:///test_orm.db")

In [37]:
from sqlalchemy import MetaData
from sqlalchemy import Table, Column
from sqlalchemy import Integer, String

metadata = MetaData()
cars_table = Table('cars', metadata,
               Column('id', Integer, primary_key=True),
               Column('name', String),
               Column('price', Integer)
             )

In [44]:
cars_table

Table('cars', MetaData(bind=None), Column('id', Integer(), table=<cars>, primary_key=True, nullable=False), Column('name', String(), table=<cars>), Column('price', Integer(), table=<cars>), schema=None)

In [45]:
print(cars_table.select())

SELECT cars.id, cars.name, cars.price 
FROM cars


In [46]:
metadata.create_all(engine)

In [50]:
from sqlalchemy import select

In [47]:
conn = engine.connect()
conn.execute(cars_table.insert(), [
        {'id': cars[0][0], 'name': cars[0][1], 'price': cars[0][2]},
        {'id': cars[1][0], 'name': cars[1][1], 'price': cars[1][2]}
])
conn.close()

In [54]:
# print(cars_table.select([cars_table]))

In [52]:
conn = engine.connect()
select_stmt = select([cars_table])
result = conn.execute(select_stmt)
for row in result:
    print(row)
conn.close()

(1, 'Audi', 52642)
(2, 'Mercedes', 57127)
